### **Data Reading (orders)**

In [0]:
df_orders = spark.read.format('delta')\
    .load('abfss://bronze@databrickseterjed.dfs.core.windows.net/orders')
df_orders.display()

### **Data Transformations**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

df_orders = df_orders.drop('_rescued_data')\
            .withColumn('order_date', to_timestamp(col('order_date')))\
            .withColumn('year', year(col('order_date')))

### **Data Writing**

In [0]:
df_orders.write.format('delta').mode('append').save('abfss://silver@databrickseterjed.dfs.core.windows.net/orders')

### **Data Reading(Customers)**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df_customers = spark.read.format('delta').load('abfss://bronze@databrickseterjed.dfs.core.windows.net/customers')
df_customers.display()

### **Data Transformations**

In [0]:
df_customers = df_customers.drop('_rescued_data')\
       .withColumn('domain', split(col('email'), '@')[1])\
       .withColumn('full_name', concat(col('first_name'), lit(' '), col('last_name')))

### **Data Writing**

In [0]:
df_customers.write.mode('overwrite').format('delta').save('abfss://silver@databrickseterjed.dfs.core.windows.net/customers')

### **Data Reading (products)**

In [0]:
df_products = spark.read.format('delta').load('abfss://bronze@databrickseterjed.dfs.core.windows.net/products')
df_products.display()

### **Data transformations**

In [0]:
from pyspark.sql.functions import col, round, when

df_products = df_products.drop('_rescued_data')\
    .withColumn('price_band',
                    when(col('price') < 500, 'Low')\
                    .when(col('price') < 1500, 'Medium')\
                    .otherwise('High'))\
    .withColumn('discount_price', round(col('price') * 0.9, 2))
df_products.display()


### **Data Writing**

In [0]:
df_products.write.format('delta').mode('overwrite').save('abfss://silver@databrickseterjed.dfs.core.windows.net/products')

### **Read, transform and write region table**

In [0]:
df_regions = spark.read.table('databricks_cat.bronze.regions')
df_regions.display()

In [0]:
df_regions.drop('_rescued_data')

In [0]:
df_regions.write.format('delta').mode('overwrite').save('abfss://silver@databrickseterjed.dfs.core.windows.net/regions')

# Create Schema and Tables

In [0]:
%sql
DROP SCHEMA IF EXISTS databricks_cat.silver CASCADE;
CREATE SCHEMA IF NOT EXISTS databricks_cat.silver;

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_cat.silver.orders 
USING DELTA
LOCATION 'abfss://silver@databrickseterjed.dfs.core.windows.net/orders';

CREATE TABLE IF NOT EXISTS databricks_cat.silver.customers
USING DELTA
LOCATION 'abfss://silver@databrickseterjed.dfs.core.windows.net/customers';

CREATE TABLE IF NOT EXISTS databricks_cat.silver.products
USING DELTA
LOCATION 'abfss://silver@databrickseterjed.dfs.core.windows.net/products';

CREATE TABLE IF NOT EXISTS databricks_cat.silver.regions
USING DELTA
LOCATION 'abfss://silver@databrickseterjed.dfs.core.windows.net/regions';
